# Topic Modelling for Industrial's M&A Deal Ratonale's

In [1]:
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from stop_words import safe_get_stop_words
from stop_words import get_stop_words
from textblob import TextBlob

In [2]:
Industrial = pd.read_excel("D:\\ISB\\Capstone Project\\Data\\3-Industrial Deals\\3-Industrials - New.xls",sheetname='MM_DealUnformattedReport')#,skiprows=7)
for i in range(0,len(Industrial)):
    try:
        Industrial['Deal Description'][i] = Industrial['Deal Description'][i].encode('utf8')
    except:
        print i

C:\Users\Rajiv2806\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
stop_words1 = set(stopwords.words('english'))
stop_words1 = " ".join(i.encode('utf-8') for i in stop_words1).split()
stop_words2 = ['million','approximately','july','march','august','february','october','press','release','usbased','usd'
               ,'company','form','filed','undisclosed']
stop_words3 = list(get_stop_words('en'))
stop_words3 = " ".join(i.encode('utf-8') for i in stop_words3).split()
stop_words4 = list(safe_get_stop_words('unsupported language'))
stop_words5 = str(Industrial[['Target Company','Bidder Company']]) #,'Sellers','Target Ticker','Exchange:Ticker','Deal Responses','Target Company Industry']])
stop_words5 = list(pd.unique(stop_words5.split()))
stop_words = list(pd.unique(stop_words1+stop_words2+stop_words3+stop_words4+stop_words5))

stemming1 = WordNetLemmatizer()    #stemming2 = PorterStemmer()

In [13]:
unigram = []
bigram = []

for i in range(0,len(Industrial)):
    to_lowercase = Industrial['Deal Description'][i].lower()
    puncuation_removed = re.sub(r'[^\w\s]','',to_lowercase)
    newline_removed = re.sub('\n',' ',puncuation_removed)
    stopwords_removed = " ".join(j for j in newline_removed.split() if j not in stop_words)
    stemming_words1 = " ".join(stemming1.lemmatize(j) for j in stopwords_removed.split())
    numbers_removed = " ".join(re.sub("\d+","",i) for i in stemming_words1.split())
    letters_removed = " ".join(j for j in numbers_removed.split() if len(j) > 3)
    trimmed = letters_removed.strip()
    unigram.append(trimmed)
    
    a = ""
    for j in range(0,(len(trimmed.split())-1)):
        empty = trimmed.split()[j]+"_"+trimmed.split()[j+1]
        a = a+" "+empty
    a = a.strip()
    bigram.append(a)
    
data = [str(bigram[i])+" "+str(unigram[i]) for i in range(len(bigram))]

n_features = 1000

# tf or count matrix
# for topic modelling LDA can only use raw term counts because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

# tf-idf matrix
# for topic modelling NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

n_topics = 8 #input("No of topics:")

nmf = NMF(n_components=n_topics,random_state=1,alpha=.1,l1_ratio=.5,init='nndsvd')
nmf = nmf.fit(tfidf)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
lda = lda.fit(tf)

# print "The Shape of NMF Matrix:",nmf.components_.shape
# print "The Shape of LDA Matrix:",lda.components_.shape

def display_topics(model, feature_names, no_top_words):
    if str(model)[0:3] == "NMF":
        print "NMF MODEL"
    elif str(model)[0:3] == "Lat":
        print "LDA MODEL"
        
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx+1)
        print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
    print "="*100
    
no_top_words = 15

print "Topics For Industrial's"
print "="*100

display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tfidf_feature_names, no_top_words)

Topics For Industrial's
NMF MODEL
Topic 1:
product service acquisition solution industrial industry corporation technology equipment transaction engaged acquired business market manufacturer
Topic 2:
share exhibit headquartered expected transaction price closing corporation condition agreement offer cash termination approval acquire
Topic 3:
logistics transportation service logistics_service freight providing provider engaged_providing transportation_service trucking transportation_logistics service_undisclosed service_acquired shipping transport
Topic 4:
packaging corrugated paper container packaging_product packaging_solution graphic plastic food paper_packaging product manufacturer display flexible consumer
Topic 5:
capital partner equity private_equity firm private equity_firm capital_partner firm_acquired partner_private undisclosed portfolio acquired undisclosed_consideration management
Topic 6:
waste service disposal recycling waste_management environmental management management

In [16]:
topic_output_lda = np.column_stack((data,lda.transform(tf)))
topic_output_nmf = np.column_stack((data,nmf.transform(tfidf)))
df1 = pd.DataFrame(topic_output_lda,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df2 = pd.DataFrame(topic_output_nmf,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df1["LDA Topic"] = df1[df1.columns[1:9]].idxmax(axis=1)
df2["NMF Topic"] = df2[df2.columns[1:9]].idxmax(axis=1)

Industrial["NMF TopicID"] = df2[df2.columns[1:9]].idxmax(axis=1)
Industrial["LDA TopicID"] = df1[df1.columns[1:9]].idxmax(axis=1)

Industrial.to_csv("D:\\ISB\\Capstone Project\\Data\\3-Industrial Deals\\Industrials - Topics.csv",index=False,encoding='utf-8')

## Summary:

Topic modelling on Industrial acquisitions is done using the LDA and NMF methods.

Results for the the Deal rationale, for 8 topics of both the models are:

**Topics from Deal Description are:**

**NMF MODEL**
Topic 1:product service acquisition solution industrial industry corporation technology equipment transaction engaged acquired business market manufacturer –product expansion

Topic 2:share exhibit headquartered expected transaction price closing corporation condition agreement offer cash termination approval acquire –deal related

Topic 3:logistics transportation service logistics_service freight providing provider engaged_providing transportation_service trucking transportation_logistics service_undisclosed service_acquired shipping transport –transportation indus

Topic 4:packaging corrugated paper container packaging_product packaging_solution graphic plastic food paper_packaging product manufacturer display flexible consumer –packaging industry

Topic 5:capital partner equity private_equity firm private equity_firm capital_partner firm_acquired partner_private undisclosed portfolio acquired undisclosed_consideration management –financial/stock related terms 

Topic 6:waste service disposal recycling waste_management environmental management management_service collection environmental_service providing solid transfer engaged_providing treatment –recycling industry

Topic 7:steel metal tube aluminum processing product metal_product corporation holding alloy dynamic fabrication employee december holding_corporation –metals industry

Topic 8:aerospace precision component defense manufacturing aircraft aerospace_defense group industry fastener aviation metal aerospace_industry military engineering –aricraft industry

**LDA MODEL**

Topic 1:service waste security holding exhibit energy logistics environmental recycling terminal management acquisition transaction partner update

Topic 2:tool communication railroad machine printing digital molding print tooling alloy real mold injection precision acquisition

Topic 3:service acquisition engaged acquired transaction source link solution source_link product corporation technology undisclosed industrial customer

Topic 4:product packaging acquisition manufacturer acquired manufacturing transaction steel metal industry undisclosed_consideration source_link link source undisclosed

Topic 5:capital equity private acquired firm private_equity partner undisclosed equity_firm undisclosed_consideration service acquisition portfolio management capital_partner

Topic 6:transaction share expected headquartered corporation exhibit condition closing price cash acquire update acquisition agreement listed

Topic 7:instrument measurement laser safe technology scientific incorporated fleet instrumentation test research science testing process life

Topic 8:industry product acquisition transaction manufacturer business electrical electric listed furniture source source_link link facility expected